In [10]:
import os,sys,subprocess,glob,cftime,importlib,pickle,itertools
from datetime import datetime
import xarray as xr
import numpy as np
import pandas as pd
sys.path.append('../')

from ensembles.ensemble_GKLT import ensemble_GKLT,get_weight_for_selection

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--project_path", type=str)
parser.add_argument("--experiment_identifiers", nargs='+', default=[f"c{i}" for i in range(1,6)] + [f"p{i}" for i in range(1,6)] + ['c1_initial', 'p1_initial'])
parser.add_argument("--overwrite", action='store_true')
command_line_arguments = parser.parse_args()


In [2]:
project_path = '../REA_heat_wEU_JJA'
experiment_identifiers = ['c7_wet']

In [3]:

sys.path.append(project_path)
from experiment_configuration.experiment import experiment

for experiment_identifier in experiment_identifiers:
    print(experiment_identifier)
    break


c7_wet


In [4]:

    exp = experiment(importlib.import_module(f"experiment_configuration.{experiment_identifier}").config)

    naming_d = {
        "project": 'REA_output',
        "product": exp.product_name,
        "institute": 'NCAR',
        "model": 'CESM2',
        "experiment" : f"{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}",
        "realm": "meta",
    }
    out_dir = '/'.join([exp.dir_work] + [v for k,v in naming_d.items()])


In [54]:
todo_tables = {}
steps = np.arange(0,exp.n_steps,1,'int')
for step in steps:
    todo_tables[step] = pd.read_table(f"{self._exp.dir_work}/GKLT/{self._exp.experiment_name}/book_keeping/step{step}.csv", sep=',')
    todo_tables[step]['parent_identifier'] = [p.split('/')[-1] for p in todo_tables[step]['parent_path']]

In [84]:
sim_names = []
for i in todo_tables[step].index:
    sim_name = exp.experiment_identifier + '.'
    sim_name += todo_tables[steps[-1]].loc[i, 'case_identifier'].split('_')[-1]
    parent_name = todo_tables[steps[-1]].loc[i, 'parent_identifier']
    for step in steps[::-1][1:]:
        sim_name += '.' + todo_tables[step].loc[todo_tables[step].case_identifier == parent_name, 'case_identifier'].values[0].split('_')[-1]
        parent_name = todo_tables[step].loc[todo_tables[step].case_identifier == parent_name, 'parent_identifier'].values[0]
    sim_names.append(sim_name)
sim_names = np.array(sim_names)
sim_names

array(['c7_wet.000.000.000.000.000.001.001.001.001.001.001.001.001.001.001.001.002.002',
       'c7_wet.001.001.001.002.002.003.001.001.001.001.001.001.001.001.001.001.002.002',
       'c7_wet.002.002.002.003.003.004.002.002.002.002.002.002.002.002.001.001.002.002',
       'c7_wet.003.003.003.004.004.005.003.003.004.004.004.004.004.004.003.003.004.004',
       'c7_wet.004.004.004.005.005.006.004.004.005.005.005.005.005.005.004.004.005.004',
       'c7_wet.005.005.005.006.006.007.005.004.005.005.005.005.005.005.004.004.005.004',
       'c7_wet.006.006.006.007.007.008.006.005.006.006.006.006.006.006.005.005.006.005',
       'c7_wet.007.007.007.008.008.009.007.006.007.007.007.007.007.007.006.006.007.006',
       'c7_wet.008.008.008.009.009.010.008.007.009.009.008.008.008.007.006.006.007.006',
       'c7_wet.009.009.008.009.009.010.008.007.009.009.008.008.008.007.006.006.007.006',
       'c7_wet.010.010.009.010.010.011.009.008.010.009.008.008.008.007.006.006.007.006',
       'c7_wet.011.01

In [95]:
sim_name = sim_names[49]
realm = 'atm'
h_identifier = 'h1'
variable = 'TREFHT'
preprocessing_module = importlib.import_module(f"data_extraction.preprocessing.regional_average_TREFHT")
preprocessor = preprocessing_module.preprocessor
end_step = 18

In [96]:
exp.dir_archive_post

'/work/bb1152/u290372/cesm215_archive/GKLT/wEU.TREFHT.05-31.5x18.ssp370-2025-wet.k0p05.s7000'

In [ ]:
l = []
for step,sim_identifier_in_step in enumerate(sim_name.split('.')[1:]):
    h_files = glob.glob(f"{exp.dir_archive_post}/step{step}/{exp.experiment_identifier}_{sim_identifier_in_step}/{realm}/hist/*{h_identifier}*.nc")
    if len(h_files) == 1:
        with xr.open_mfdataset(h_files[0], preprocess=preprocessor) as nc:
            l.append(nc[variable])
if len(l) == end_step:
    x = xr.concat(l, dim='time')
    
return x, nc.attrs

In [ ]:
todos = [['/'.join(sim_name.split('/')[:step])] for step in range(1,end_step+1)]
l = []
for step in range(1,end_step+1):
    _sim_name_of_step_ = '/'.join(sim_name.split('/')[:step])
    h_files = glob.glob(f"{exp.dir_archive_post}/{_sim_name_of_step_}/{realm}/hist/*{h_identifier}*.nc")
    print(h_files)
    if len(h_files) == 1:
        with xr.open_mfdataset(h_files[0], preprocess=preprocessor) as nc:
            print(nc)
            l.append(nc[variable])

if len(l) == end_step:
    x = xr.merge(l)[variable].sortby('time')

return x, nc.attrs

In [141]:
ens = ensemble_GKLT(exp)

In [168]:
obs = xr.open_mfdataset(f"{exp.dir_work}/REA_output/{exp.product_name}/NCAR/CESM2/{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}/day/atmos/tas-reg/*/*", concat_dim='sim', combine='nested')['tas'].load()

x = xr.DataArray(
    obs.values.reshape((-1, exp.n_steps, exp.n_days)),
    dims=['sim','step','time'],
    coords=dict(sim=obs.sim, step=np.arange(0,exp.n_steps,1,'int'), time=np.arange(0,exp.n_days,1,'int'))
)
time_sum_over_each_step = x.mean('time')

scores = np.exp(exp.k * time_sum_over_each_step)


In [169]:

mean_scores = np.array([
            pd.read_table(f"{self._exp.dir_work}/GKLT/{self._exp.experiment_name}/book_keeping/step{step}_evaluation.csv", sep=',')['score'].values.mean()
            for step in range(self._exp.n_steps)
        ])


In [170]:
i=0

In [171]:
time_sum_over_each_step

<xarray.DataArray (sim: 100, step: 18)> Size: 7kB
array([[ 19.77085 ,  17.767315,  17.334248, ...,  18.830006,  22.137396,
         22.056396],
       [ 19.75501 ,  17.716162,  17.543743, ...,  18.830006,  22.137396,
         22.056396],
       [ 19.743567,  17.735973,  17.801128, ...,  18.830006,  22.137396,
         22.056396],
       ...,
       [290.6975  , 293.54102 , 290.4439  , ..., 298.191   , 290.42206 ,
        293.6573  ],
       [290.63977 , 293.49295 , 291.98236 , ..., 295.5403  , 298.1585  ,
        292.90115 ],
       [290.66122 , 293.6634  , 291.40912 , ..., 297.58875 , 297.805   ,
        295.53955 ]], dtype=float32)
Coordinates:
  * sim      (sim) <U78 31kB 'c7_wet.000.000.000.000.000.001.001.001.001.001....
  * step     (step) int64 144B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17

In [172]:
scores[i,:-1]

<xarray.DataArray (step: 17)> Size: 68B
array([2.687315 , 2.4311533, 2.3790767, 3.165168 , 2.6106477, 2.6040776,
       2.3804462, 2.4031496, 2.4705977, 2.8679743, 2.6793954, 3.281342 ,
       3.3270147, 2.8252707, 2.6136787, 2.5638251, 3.024875 ],
      dtype=float32)
Coordinates:
    sim      <U78 312B 'c7_wet.000.000.000.000.000.001.001.001.001.001.001.00...
  * step     (step) int64 136B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16

In [173]:
mean_scores

array([2.3296178 , 2.53357052, 2.58302019, 2.63574299, 2.55188286,
       2.62688054, 2.72305259, 2.77497038, 2.84103618, 2.89851221,
       2.97413476, 2.99599872, 3.04055294, 3.04018773, 3.0000418 ,
       2.97767777, 2.89999853, 2.79642199])

In [174]:
np.product(mean_scores[:-1]) / (np.product(scores[i,:-1]) * exp.n_members)

<xarray.DataArray ()> Size: 8B
array(0.01574955)
Coordinates:
    sim      <U78 312B 'c7_wet.000.000.000.000.000.001.001.001.001.001.001.00...

In [151]:

weight_from_algo = obs.mean('time').copy() * np.nan
weight_from_algo[:] = np.array(
    [
        np.product(mean_scores[:-1]) / (np.product(scores[i,:-1]) * exp.n_members)
        for i in range(exp.n_members)
    ]
)


/tmp/ipykernel_3415572/3888587136.py:3: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  [
/home/u/u290372/.conda/envs/py_gmlnet/lib/python3.9/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/tmp/ipykernel_3415572/3888587136.py:3: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  [
/home/u/u290372/.conda/envs/py_gmlnet/lib/python3.9/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/tmp/ipykernel_3415572/3888587136.py:3: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  [
/home/u/u290372/.conda/envs/py_gmlnet/lib/python3.9/site-packages/numpy/core/fromnumeric.

In [ ]:

prob = obs.mean('time').copy() * np.nan
prob[:] = np.array([np.sum((abs >= a).astype(float) * weight_from_algo) for a in abs]) 


In [145]:
ens._trajectories['obs']

<xarray.DataArray 'tas' (sim: 100, time: 90)> Size: 36kB
array([[291.6583 , 294.3322 , 294.92963, ..., 293.91486, 294.78757,
        297.01752],
       [291.64322, 294.34174, 294.93848, ..., 293.91486, 294.78757,
        297.01752],
       [291.63977, 294.32065, 294.93817, ..., 293.91486, 294.78757,
        297.01752],
       ...,
       [291.97498, 289.9892 , 290.0773 , ..., 293.8344 , 292.31677,
        289.91586],
       [291.97662, 289.99066, 290.02835, ..., 292.5157 , 290.91443,
        289.81775],
       [291.98206, 289.98743, 290.06064, ..., 296.43317, 296.0976 ,
        295.08725]], dtype=float32)
Coordinates:
  * sim      (sim) <U78 31kB 'c7_wet.000.000.000.000.000.001.001.001.001.001....
  * time     (time) datetime64[ns] 720B 2025-06-01 2025-06-02 ... 2025-08-29

In [ ]:
ens.evaluate_weights_and_probabilities()

/home/u/u290372/.conda/envs/py_gmlnet/lib/python3.9/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/u/u290372/.conda/envs/py_gmlnet/lib/python3.9/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/u/u290372/.conda/envs/py_gmlnet/lib/python3.9/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/u/u290372/.conda/envs/py_gmlnet/lib/python3.9/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/u/u290372/.conda/envs/py_gmlnet/lib/python3.9/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **pa

In [ ]:
    ens.evaluate_weights_and_probabilities()

    xr.Dataset({'probability':ens._prob}).to_netcdf(f"{out_dir}/probability_season_{naming_d['experiment']}.nc")
    xr.Dataset({'weight':ens._weight_from_algo}).to_netcdf(f"{out_dir}/weight_season_{naming_d['experiment']}.nc")